In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime

In [9]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import or_

Base_declare = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

In [3]:
# create an engine
engine = create_engine("sqlite:///db/wine.sqlite")
conn = engine.connect()

In [4]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [5]:
# View all of the classes/tables that automap found
Base.classes.keys()

['D3_table', 'map_wine_table', 'wine_table']

In [6]:
# Save references to the table
Wine = Base.classes.wine_table

In [7]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [8]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

# inspecting the columns
columns = inspector.get_columns('wine_table')   
for column in columns:
    print(column["name"], column["type"])

id INTEGER
country VARCHAR(255)
description VARCHAR(1000)
points FLOAT
price FLOAT
province VARCHAR(255)
region_1 VARCHAR(255)
region_2 VARCHAR(255)
variety VARCHAR(255)
winery VARCHAR(255)


In [13]:
# inspecting the rows Measurement Table 
# engine.execute('select * from wine_table LIMIT 5').fetchall()  

In [14]:
# Design a query to show how many wineries are available in this dataset?
session.query(func.count(Wine.winery)).all()

[(137230)]

# Wineries that got the most reviews - will need that for ML part. 

In [17]:
# TODO: save this query for ML Part 
# TODO: visualize number of reviews ONLY per country on the scatter plot 
# Then point out the wineries that got the most reviews 

# What are the most active winaries? Listing the stations and the 
# counts in descending order.
session.query(Wine.winery, Wine.country, func.count(Wine.winery)).\
    group_by(Wine.winery).\
    order_by(func.count(Wine.winery).desc()).all()

[('Williams Selyem', 'US', 371),
 ('Testarossa', 'US', 274),
 ('DFJ Vinhos', 'Portugal', 249),
 ('Chateau Ste. Michelle', 'US', 225),
 ('Columbia Crest', 'US', 216),
 ('Kendall-Jackson', 'US', 216),
 ('Concha y Toro', 'Chile', 214),
 ('Trapiche', 'Argentina', 201),
 ('Bouchard Père & Fils', 'France', 192),
 ('De Loach', 'US', 189),
 ('Joseph Drouhin', 'France', 185),
 ('Kenwood', 'US', 183),
 ('Cameron Hughes', 'US', 172),
 ("D'Arenberg", 'Australia', 153),
 ('Dry Creek Vineyard', 'US', 153),
 ('Louis Latour', 'France', 153),
 ('Morgan', 'US', 153),
 ('Concannon', 'US', 151),
 ('Robert Mondavi', 'US', 151),
 ('Martin Ray', 'US', 149),
 ('Errazuriz', 'Chile', 148),
 ('Wines & Winemakers', 'Portugal', 148),
 ("L'Ecole No. 41", 'US', 144),
 ('Iron Horse', 'US', 142),
 ('Montes', 'Chile', 142),
 ('Renwood', 'US', 141),
 ('Santa Rita', 'Chile', 141),
 ('Waterbrook', 'US', 141),
 ('Calera', 'US', 140),
 ('Hogue', 'US', 139),
 ('Yalumba', 'Australia', 139),
 ('Gary Farrell', 'US', 138),
 ('Sa

In [18]:
# Reviews by country 
session.query(Wine.country, func.count(Wine.description)).\
    group_by(Wine.country).\
    order_by(func.count(Wine.country).desc()).all()

[('US', 62139),
 ('Italy', 18784),
 ('France', 14785),
 ('Spain', 8160),
 ('Chile', 5766),
 ('Argentina', 5587),
 ('Australia', 4894),
 ('Portugal', 4176),
 ('New Zealand', 3070),
 ('Austria', 2483),
 ('Germany', 2347),
 ('South Africa', 2237),
 ('Greece', 872),
 ('Israel', 610),
 ('Hungary', 230),
 ('Canada', 194),
 ('Romania', 139),
 ('Uruguay', 85),
 ('Croatia', 83),
 ('Slovenia', 81),
 ('Bulgaria', 77),
 ('Moldova', 71),
 ('Mexico', 63),
 ('Turkey', 50),
 ('Georgia', 43),
 ('Lebanon', 37),
 ('Cyprus', 31),
 ('Brazil', 25),
 ('Macedonia', 16),
 ('Serbia', 14),
 ('Morocco', 12),
 ('Luxembourg', 9),
 ('England', 8),
 ('India', 8),
 ('Lithuania', 8),
 ('Czech Republic', 6),
 ('Ukraine', 5),
 ('Bosnia and Herzegovina', 4),
 ('South Korea', 4),
 ('Switzerland', 4),
 ('China', 3),
 ('Slovakia', 3),
 ('Albania', 2),
 ('Japan', 2),
 ('Montenegro', 2),
 ('US-France', 1)]

In [102]:
# Query the wineraies with avg point and avg price, along with country 
# and province 
for_df = session.query(Wine.winery, Wine.country, Wine.province, func.avg(Wine.points), func.avg(Wine.price)).\
        group_by(Wine.winery).\
        order_by(func.avg(Wine.points).desc()).all()
# print(for_df)

In [ ]:
# Get how many records each country has and pick 12 
session.query(Wine.country, func.count(Wine.points)).\
    group_by(Wine.country).\
    order_by(func.count(Wine.country).desc()).all()

In [41]:
# Get the data for country, price, quality for D3 chart
for_d3 = session.query(Wine.country, Wine.price, Wine.points).\
filter(or_(Wine.country == "US", Wine.country == "France", 
Wine.country == "Italy", Wine.country == "Spain",
Wine.country == "Chile", Wine.country == "Argentina",
Wine.country == "Australia", Wine.country == "Portugal",
Wine.country == "New Zealand", Wine.country == "Austria",
Wine.country == "Germany", Wine.country == "England"))\
.all()

[('US', 235.0, 96.0), ('Spain', 110.0, 96.0), ('US', 90.0, 96.0), ('US', 65.0, 96.0), ('France', 66.0, 95.0), ('Spain', 73.0, 95.0), ('Spain', 65.0, 95.0), ('Spain', 110.0, 95.0), ('US', 65.0, 95.0), ('US', 60.0, 95.0), ('Italy', 80.0, 95.0), ('US', 48.0, 95.0), ('US', 48.0, 95.0), ('France', 90.0, 95.0), ('US', 185.0, 95.0), ('US', 90.0, 95.0), ('US', 325.0, 95.0), ('Spain', 80.0, 95.0), ('France', 290.0, 95.0), ('US', 75.0, 95.0), ('US', 24.0, 95.0), ('Spain', 79.0, 95.0), ('Spain', 220.0, 95.0), ('US', 60.0, 95.0), ('US', 45.0, 95.0), ('New Zealand', 57.0, 94.0), ('US', 62.0, 94.0), ('US', 105.0, 94.0), ('US', 60.0, 94.0), ('US', 60.0, 94.0), ('US', 37.0, 90.0), ('France', 22.0, 90.0), ('US', 42.0, 90.0), ('Italy', 135.0, 90.0), ('France', 60.0, 90.0), ('Italy', 29.0, 90.0), ('Italy', 23.0, 90.0), ('Italy', 29.0, 90.0), ('Spain', 17.0, 90.0), ('Spain', 26.0, 90.0), ('US', 55.0, 90.0), ('Italy', 39.0, 90.0), ('France', 69.0, 90.0), ('Italy', 30.0, 90.0), ('Italy', 90.0, 90.0), ('US',

In [100]:
# Get the data for country, price, quality for D3 chart
q = session.query(Wine.country, Wine.price, Wine.points).\
filter(Wine.country == "Greece").\
limit(100).all()

In [101]:
# Convert the results to the lists of lists
df = pd.DataFrame(q, columns=['Country', 'Price', 'Quality'])
final = df.values.tolist()

[['Greece', 24.0, 89.0],
 ['Greece', 21.0, 89.0],
 ['Greece', 18.0, 89.0],
 ['Greece', 17.0, 89.0],
 ['Greece', 17.0, 89.0],
 ['Greece', 20.0, 90.0],
 ['Greece', 20.0, 90.0],
 ['Greece', 13.0, 89.0],
 ['Greece', 19.0, 88.0],
 ['Greece', 28.0, 90.0],
 ['Greece', 12.0, 88.0],
 ['Greece', 20.0, 89.0],
 ['Greece', 25.0, 87.0],
 ['Greece', 20.0, 89.0],
 ['Greece', 30.0, 89.0],
 ['Greece', 21.0, 89.0],
 ['Greece', 17.0, 87.0],
 ['Greece', 25.0, 87.0],
 ['Greece', 18.0, 88.0],
 ['Greece', 13.0, 87.0],
 ['Greece', 10.0, 87.0],
 ['Greece', 33.0, 87.0],
 ['Greece', 15.0, 87.0],
 ['Greece', 25.0, 87.0],
 ['Greece', 30.0, 87.0],
 ['Greece', 20.0, 88.0],
 ['Greece', 23.0, 88.0],
 ['Greece', 17.0, 86.0],
 ['Greece', 13.0, 86.0],
 ['Greece', 11.0, 88.0],
 ['Greece', 30.0, 87.0],
 ['Greece', 30.0, 87.0],
 ['Greece', 13.0, 85.0],
 ['Greece', 13.0, 85.0],
 ['Greece', 16.0, 85.0],
 ['Greece', 12.0, 84.0],
 ['Greece', 39.0, 91.0],
 ['Greece', 35.0, 91.0],
 ['Greece', 22.0, 91.0],
 ['Greece', 25.0, 90.0],


In [40]:
# Convert the data into a dataframe
d3_df = pd.DataFrame(for_d3, columns=['Country', 'Price', 'Quality'])
d3_df.head(2)

,Country,Price,Quality
0,US,235.0,96.0
1,Spain,110.0,96.0


In [50]:
# Convert floats to int
d3_df["Price"]= d3_df["Price"].astype(int) 
d3_df["Quality"]= d3_df["Quality"].astype(int) 

d3_df.head(2)

,Country,Price,Quality
0,US,235,96
1,Spain,110,96


In [15]:
# Inspecting the data province
session.query(Wine.country, Wine.province).\
    filter(Wine.country == "US").\
    group_by(Wine.province).all()

[('US', 'America'),
 ('US', 'Arizona'),
 ('US', 'California'),
 ('US', 'Colorado'),
 ('US', 'Connecticut'),
 ('US', 'Idaho'),
 ('US', 'Iowa'),
 ('US', 'Kentucky'),
 ('US', 'Massachusetts'),
 ('US', 'Michigan'),
 ('US', 'Missouri'),
 ('US', 'Nevada'),
 ('US', 'New Jersey'),
 ('US', 'New Mexico'),
 ('US', 'New York'),
 ('US', 'North Carolina'),
 ('US', 'Ohio'),
 ('US', 'Oregon'),
 ('US', 'Pennsylvania'),
 ('US', 'Texas'),
 ('US', 'Vermont'),
 ('US', 'Virginia'),
 ('US', 'Washington'),
 ('US', 'Washington-Oregon')]

In [22]:
# Checking the wineraies in Cali?
session.query(Wine.province, Wine.winery, func.count(Wine.description)).\
        filter(Wine.province == "California").\
        group_by(Wine.winery).\
        order_by(func.count(Wine.description).desc()).all()

[('California', 'Williams Selyem', 371),
 ('California', 'Testarossa', 274),
 ('California', 'Kendall-Jackson', 216),
 ('California', 'De Loach', 189),
 ('California', 'Kenwood', 183),
 ('California', 'Dry Creek Vineyard', 153),
 ('California', 'Morgan', 153),
 ('California', 'Concannon', 151),
 ('California', 'Robert Mondavi', 151),
 ('California', 'Martin Ray', 149),
 ('California', 'Cameron Hughes', 147),
 ('California', 'Iron Horse', 142),
 ('California', 'Renwood', 141),
 ('California', 'Calera', 140),
 ('California', 'Gary Farrell', 138),
 ('California', 'Rodney Strong', 128),
 ('California', 'Wellington', 127),
 ('California', 'Adelaida', 122),
 ('California', 'Novy', 121),
 ('California', 'Thomas Fogarty', 121),
 ('California', 'Dutton-Goldfield', 120),
 ('California', 'Brander', 118),
 ('California', 'Fess Parker', 116),
 ('California', 'Beringer', 115),
 ('California', 'J. Lohr', 115),
 ('California', 'Peachy Canyon', 115),
 ('California', 'Bernardus', 112),
 ('California', '

# Dumping Data into sqlite 

In [74]:
# Save the list in sqllite?
# Creating a Class & table 
class D3(Base_declare):
    __tablename__ = 'D3_table'
    id = Column(Integer, primary_key=True)
    country = Column(String(255))
    price = Column(Integer)
    quality = Column(Integer)

/anaconda3/envs/PythonData/lib/python3.7/site-packages/sqlalchemy/ext/declarative/clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.D3, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)


In [75]:
Base_declare.metadata.create_all(conn)

In [76]:
d3_df.to_sql(name='D3_table', con=conn, if_exists='append', index=False)

In [82]:
# Inspecting the Table to make sure the table is there
inspector = inspect(engine)
inspector.get_table_names()

['D3_table', 'map_wine_table', 'wine_table']

In [81]:
# Checking the data 
engine.execute('SELECT * FROM D3_table LIMIT 20').fetchall()

[(1, 'US', 235, 96),
 (2, 'Spain', 110, 96),
 (3, 'US', 90, 96),
 (4, 'US', 65, 96),
 (5, 'France', 66, 95),
 (6, 'Spain', 73, 95),
 (7, 'Spain', 65, 95),
 (8, 'Spain', 110, 95),
 (9, 'US', 65, 95),
 (10, 'US', 60, 95),
 (11, 'Italy', 80, 95),
 (12, 'US', 48, 95),
 (13, 'US', 48, 95),
 (14, 'France', 90, 95),
 (15, 'US', 185, 95),
 (16, 'US', 90, 95),
 (17, 'US', 325, 95),
 (18, 'Spain', 80, 95),
 (19, 'France', 290, 95),
 (20, 'US', 75, 95)]

# Exploratory Analysis

In [120]:
# Explore wineries 
session.query(Wine.winery, Wine.country, Wine.province, Wine.variety, Wine.points, Wine.price).\
    group_by(Wine.winery).\
    order_by(func.avg(Wine.points).desc()).all()

[('Sloan', 'US', 'California', 'Cabernet Blend', 100.0, 245.0),
 ('Mascarello Giuseppe e Figlio',
  'Italy',
  'Piedmont',
  'Nebbiolo',
  99.0,
  175.0),
 ('Clos de Tart', 'France', 'Burgundy', 'Pinot Noir', 98.0, 319.0),
 ('Au Sommet', 'US', 'California', 'Cabernet Sauvignon', 97.0, 250.0),
 ('Cardinale', 'US', 'California', 'Bordeaux-style Red Blend', 93.0, 300.0),
 ('Domaine Bruno Clair', 'France', 'Burgundy', 'Pinot Noir', 97.0, 281.0),
 ('Gandona', 'US', 'California', 'Cabernet Sauvignon', 97.0, 190.0),
 ('Ovid', 'US', 'California', 'Bordeaux-style Red Blend', 97.0, 195.0),
 ('Vieux Château Certan',
  'France',
  'Bordeaux',
  'Bordeaux-style Red Blend',
  97.0,
  130.0),
 ('Château Climens',
  'France',
  'Bordeaux',
  'Bordeaux-style White Blend',
  95.0,
  135.0),
 ('Harlan Estate',
  'US',
  'California',
  'Bordeaux-style Red Blend',
  97.0,
  150.0),
 ('Araujo', 'US', 'California', 'Cabernet Sauvignon', 96.0, 185.0),
 ('Bryant Family', 'US', 'California', 'Cabernet Sauvigno

In [107]:
session.query(Wine.winery, func.count(Wine.description)).\
     filter(Wine.winery == "Sloan").all()
    

[('Sloan', 2)]